# Database Administration

This notebook provides database management utilities:
- Initialize database
- Test connection
- View database statistics
- Cleanup utilities

In [ ]:
# Load helpers
%run /home/jovyan/work/system/helpers/01_Database.ipynb
%run /home/jovyan/work/system/helpers/04_Display.ipynb

## Test Database Connection

In [ ]:
print_header("Database Connection Test")

if test_connection():
    print("✅ Database connection successful")
    
    # Show connection details
    data = [
        ['Server', DB_CONFIG['server']],
        ['Database', DB_CONFIG['database']],
        ['Port', DB_CONFIG['port']],
        ['User', DB_CONFIG['username']]
    ]
    display_status_table(data, headers=['Property', 'Value'])
    
    # Get database version
    version = execute_scalar("SELECT @@VERSION")
    print(f"\nSQL Server Version:")
    print(version[:100] + "...")
else:
    print("❌ Database connection failed")
    print("\nTroubleshooting:")
    print("1. Check if SQL Server container is running: docker ps")
    print("2. Verify environment variables in .env file")
    print("3. Try initializing the database (next cell)")

## Initialize Database

**Warning:** This will recreate all tables!

In [ ]:
# Uncomment the next line to initialize/reset the database
# init_database()

## Database Statistics

In [ ]:
print_header("Database Statistics")

# Table row counts
tables = ['irp_cycle', 'irp_stage', 'irp_step', 'irp_step_run', 'irp_batch', 'irp_job']
stats = []

for table in tables:
    try:
        count = execute_scalar(f"SELECT COUNT(*) FROM {table}")
        stats.append([table, count])
    except:
        stats.append([table, 'N/A'])

display_status_table(stats, headers=['Table', 'Row Count'])

# Active cycle info
active = get_active_cycle()
if active:
    print(f"\n✅ Active Cycle: {active['cycle_name']}")
else:
    print("\n⚠️  No active cycle")

## View Recent Activity

In [ ]:
print_header("Recent Step Runs")

query = """
SELECT TOP 10
    c.cycle_name,
    CONCAT(sg.stage_num, '.', st.step_num) as step,
    st.step_name,
    sr.status,
    sr.started_ts,
    sr.completed_ts
FROM irp_step_run sr
INNER JOIN irp_step st ON sr.step_id = st.id
INNER JOIN irp_stage sg ON st.stage_id = sg.id
INNER JOIN irp_cycle c ON sg.cycle_id = c.id
ORDER BY sr.started_ts DESC
"""

df = execute_query(query)
if not df.empty:
    display_dataframe(df, max_rows=10)
else:
    print("No step runs found")

## Cleanup Utilities

In [ ]:
# Clear failed step runs (uncomment to execute)
# rows = execute_command("DELETE FROM irp_step_run WHERE status = 'failed'")
# print(f"Deleted {rows} failed step runs")

# Clear all step runs but keep steps (uncomment to execute)
# rows = execute_command("DELETE FROM irp_step_run")
# print(f"Deleted {rows} step runs")

# Remove archived cycles older than 30 days (uncomment to execute)
# query = """DELETE FROM irp_cycle 
#           WHERE status = 'archived' 
#           AND archived_ts < DATEADD(day, -30, GETUTCDATE())"""
# rows = execute_command(query)
# print(f"Deleted {rows} old archived cycles")